In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 1

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [5]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=True)

Attack dataset was already established previosly, loading dataset from "pickle/tinyimagenet/mobilenetv2/attack_train.p".


In [6]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 1.7030,  0.6128,  0.5038]],  [[ 2.3362,  2.3186,  2.0089]],  [[ 3.3956,  3.2006,  2.8834]],  [[ 2.1359,  0.7500,  0.7089]],  [[ 2.7621,  2.3960,  2.3026]],  [[ 2.2115,  0.7964,  0.0623]],  [[ 2.6597,  2.1870,  1.3616]],  [[ 1.3809,  1.0832,  1.0585]],  [[ 4.5301,  1.8755,  1.6888]],  [[ 1.0893,  0.6552,  0.0521]],  [[ 1.1781,  0.9118,  0.8309]],  [[ 2.6503,  1.7803,  1.2421]],  [[ 2.8521,  1.2624,  1.1395]],  [[ 1.3660,  0.9558,  0.6079]],  [[ 1.1571,  0.7434,  0.5456]],  [[ 1.3294,  0.2896,  0.2694]],  [[ 3.7332,  1.8675,  1.8548]],  [[ 1.6870,  0.9661,  0.8747]],  [[ 0.9435,  0.8471,  0.5223]],  [[ 0.7366,  0.5973,  0.4091]],  [[ 3.2018,  2.9480,  2.2033]],  [[ 1.8640,  1.1861,  1.0658]],  [[ 2.4932,  1.9381,  1.5768]],  [[ 0.6242,  0.3315,  0.2648]],  [[ 1.8231,  1.7536,  1.7149]],  [[ 1.5350,  1.4737,  1.3058]],  [[ 3.3500,  3.1747,  2.9893]],  [[ 1.7409,  1.4165,  1.2084]],  [[ 2.2179,  1.8106,  1.8041]],  [[ 1.6116,  1.5550,  1.1991]],  [[ 1.2388,  1.1559,  0.8741]], 

### Attack Model

Simple Model

In [7]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [8]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [9]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [10]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE, test_dataset=False, standardize=True)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.9686, 0.9686, 0.9725,  ..., 0.9765, 0.9725, 0.9686], [0.9608, 0.9647, 0.9647,  ..., 0.9725, 0.9686, 0.9686], [0.9647, 0.9647, 0.9686,  ..., 0.9686, 0.9647, 0.9647], ..., [0.3843, 0.1294, 0.3294,  ..., 0.6627, 0.6980, 0.4980], [0.1333, 0.3804, 0.4118,  ..., 0.4078, 0.5020, 0.2902], [0.1569, 0.2863, 0.4353,  ..., 0.4510, 0.6549, 0.4078]],[[0.9765, 0.9765, 0.9804,  ..., 0.9843, 0.9804, 0.9765], [0.9686, 0.9725, 0.9725,  ..., 0.9804, 0.9765, 0.9765], [0.9725, 0.9725, 0.9765,  ..., 0.9765, 0.9725, 0.9725], ..., [0.3647, 0.1137, 0.3176,  ..., 0.5765, 0.6118, 0.4118], [0.1098, 0.3608, 0.4000,  ..., 0.3098, 0.4039, 0.1922], [0.1294, 0.2667, 0.4235,  ..., 0.3490, 0.5490, 0.3020]],[[0.9333, 0.9333, 0.9373,  ..., 0.9412, 0.9373, 0.9333], [0.9255, 0.9294, 0.9294,  ..., 0.9373, 0.9333, 0.9333], [0.9294, 0.9294, 0.9333,  ..., 0.9333, 0.9294, 0.9294], ..., [0.1451, 0.0000,

Attack Evaluation DL Sample

In [11]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[13.5679,  5.8385,  5.4727]],  [[11.7774,  5.1659,  4.6183]],  [[13.3103,  8.1171,  7.0456]],  [[13.3904,  7.4329,  6.8716]],  [[11.7441,  6.6519,  5.4875]],  [[14.0014,  7.6744,  7.2380]],  [[12.9404,  5.9869,  5.9537]],  [[13.9780,  6.2884,  6.0444]],  [[13.6892,  7.4124,  6.7361]],  [[12.5785,  7.3046,  6.2111]],  [[14.1772,  8.3490,  7.9208]],  [[11.6784,  5.1415,  4.5934]],  [[12.9706,  6.0249,  5.8534]],  [[15.1840,  8.7963,  7.2625]],  [[13.2971,  6.5384,  6.2782]],  [[13.0019,  6.3516,  6.1218]],  [[13.3297,  6.4383,  5.7660]],  [[11.9722,  5.6946,  4.8598]],  [[13.7635,  7.2811,  7.0462]],  [[12.5942,  7.0454,  5.9499]],  [[12.5188,  6.4181,  5.3020]],  [[12.9748,  6.8068,  6.4039]],  [[11.4711,  4.9788,  4.5095]],  [[11.8016,  5.3340,  4.8724]],  [[12.6197,  5.8138,  5.2405]],  [[12.1706,  5.5856,  5.2790]],  [[13.2897,  7.3200,  6.3253]],  [[13.3285,  7.5621,  5.6963]],  [[14.6852,  7.2786,  7.0723]],  [[12.4365,  4.8604,  4.7229]],  [[13.4672,  6.5357,  6.2461]], 

### Train Attack model

In [12]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 50000
Batchsize: 64
Inputs:tensor([[[ 2.1698e+00,  1.2757e+00,  6.7375e-01]],  [[ 1.8087e+00,  1.7401e+00,  1.4687e+00]],  [[ 1.0782e+00,  1.0232e+00,  9.5794e-01]],  [[ 2.4152e+00,  1.7452e+00,  1.2688e+00]],  [[ 1.9492e+00,  1.8756e+00,  1.6512e+00]],  [[ 1.3212e+00,  7.8907e-01,  5.8626e-01]],  [[ 5.1405e-01,  1.5642e-02, -2.5709e-02]],  [[ 1.3537e+00,  1.0993e+00,  1.0211e+00]],  [[ 1.0655e+00,  8.7155e-01,  6.2335e-01]],  [[ 2.9499e+00,  1.8178e+00,  1.5377e+00]],  [[ 1.7015e+00,  1.6227e+00,  1.0361e+00]],  [[ 2.6198e+00,  2.3277e+00,  2.2493e+00]],  [[ 5.7776e+00,  5.2099e+00,  4.9542e+00]],  [[ 2.2476e+00,  2.0817e+00,  1.9666e+00]],  [[ 1.5628e+00,  1.2597e+00,  9.7013e-01]],  [[ 2.0744e+00,  1.8343e+00,  1.7495e+00]],  [[ 1.6419e+00,  1.5372e+00,  1.2912e+00]],  [[ 1.9054e+00,  1.3794e+00,  9.3877e-01]],  [[ 1.0367e+00,  6.6082e-01,  6.2882e-01]],  [[ 2.2811e-01,  4.8819e-02,

### Evaluation

In [13]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 32.92
